In [1]:
import pandas as pd
from cassandra.cluster import Cluster

In [2]:
df=pd.read_csv('./4Assi_cass.csv')
df.head(4)

,SKU,name,description,warehouse_num
0,1,'Books','different types of books text & notes ..etc',123
1,2,'pens','different color pens of same company',456
2,3,'Laptops','different laptop of same company',789
3,4,'Computers','different types of computers',123


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SKU            7 non-null      int64 
 1   name           7 non-null      object
 2   description    7 non-null      object
 3   warehouse_num  7 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 352.0+ bytes


## create connection with cassandr

In [4]:
clstr = Cluster()
session = clstr.connect()

In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04
w04python


In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventoryInformation ( 
    SKU int, 
    name TEXT, 
    description TEXT,
    warehouse_num int, 
    PRIMARY KEY(SKU)
);
""")

In [8]:
for index, row in df.iterrows():
    session.execute(f"""
        INSERT INTO m14.inventoryInformation (SKU, description, name, warehouse_num)     
        VALUES ({row[0]}, {row[2]}, {row[1]}, {row[3]});
        """
       )



In [9]:
rows = session.execute("select (SKU, description, name, warehouse_num) from m14.inventoryInformation")
for row in rows:
    print(f"SKU={row[0][0]}, description={row[0][1]}, name={row[0][2]}, warehouse_num={row[0][3]}")

SKU=5, description=different types of student electronic pads, name=pads, warehouse_num=789
SKU=1, description=different types of books text & notes ..etc, name=Books, warehouse_num=123
SKU=2, description=different color pens of same company, name=pens, warehouse_num=456
SKU=4, description=different types of computers, name=Computers, warehouse_num=123
SKU=7, description=different sizes screen, name=monitors, warehouse_num=456
SKU=6, description=headsets, name=headsets, warehouse_num=123
SKU=3, description=different laptop of same company, name=Laptops, warehouse_num=789


In [10]:
session.execute("create index warehouIndex1 on m14.inventoryInformation(warehouse_num)")



In [11]:
rows=session.execute("select (SKU, description, name, warehouse_num) from m14.inventoryInformation where warehouse_num=123 ")
for row in rows:
    print(f"SKU={row[0][0]}, description={row[0][1]}, name={row[0][2]}, warehouse_num={row[0][3]}")


SKU=1, description=different types of books text & notes ..etc, name=Books, warehouse_num=123
SKU=4, description=different types of computers, name=Computers, warehouse_num=123
SKU=6, description=headsets, name=headsets, warehouse_num=123
